In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
from sklearn.linear_model import LassoCV

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
lasso = LassoCV(cv=5, random_state=8)
lasso.fit(X, y)

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.05620919454608497, tolerance: 0.021966098945660997
  model = cd_fast.enet_coordinate_descent_gram(
/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0725133651969827, tolerance: 0.021966098945660997
  model = cd_fast.enet_coordinate_descent_gram(
/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.07533325743443697, tolerance: 0.021966098945660997
  model = cd_fast.enet_coordinate_descent_gram(
/Users/prathmesh/

LassoCV(cv=5, random_state=8)

In [8]:
selected_mask = lasso.coef_ != 0
selected_features = list(X.columns[selected_mask])
print("Selected Features:", selected_features)

Selected Features: ['BMI', 'Fast_food_Y_N', 'LHmIU_mL', 'Follicle_No_L', 'Testosterone_Levelng_dL', 'Follicle_No_R', 'AMHng_mL', 'Skin_darkening_Y_N', 'Weight_gainY_N', 'hair_growthY_N', 'Antral_Follicle_Count', 'CycleR_I', 'Menstrual_Irregularity']


In [9]:
X = X[selected_features]
X

,BMI,Fast_food_Y_N,LHmIU_mL,Follicle_No_L,Testosterone_Levelng_dL,Follicle_No_R,AMHng_mL,Skin_darkening_Y_N,Weight_gainY_N,hair_growthY_N,Antral_Follicle_Count,CycleR_I,Menstrual_Irregularity
0,-1.339000,1.307469,-0.025653,-1.004169,0.920958,-1.120657,-0.905129,-0.983869,-1.085369,-0.945276,-0.416375,-0.961306,0.657014
1,-0.159445,-1.441998,-0.072512,-1.004169,-0.683318,-0.494685,-1.032983,-0.983869,-1.085369,-0.945276,-0.550756,-0.961306,-0.743990
2,-0.075192,1.307469,-0.076312,2.294259,0.412582,2.635177,0.174528,-0.983869,-1.085369,-0.945276,0.154743,-0.961306,1.124015
3,0.851601,-1.441998,-0.049535,-1.334012,-0.414417,-1.433643,-1.106381,-0.983869,-1.085369,-0.945276,-0.214804,-0.961306,0.657014
4,-1.170492,-1.441998,-0.075950,-1.004169,-0.028822,-0.807671,-0.860143,-0.983869,-1.085369,-0.945276,-0.248399,-0.961306,-0.276988
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,-1.528571,-0.892105,-0.050476,-0.674326,1.825076,-0.369490,-0.135636,0.200155,-1.085369,-0.311390,0.927432,-0.961306,1.124015
1537,0.683094,-1.441998,-0.058762,0.249234,-1.584392,-0.181699,-0.599226,-0.391857,0.456629,-0.311390,-1.760183,-0.359733,1.124015
1538,0.556713,-0.342211,-0.067700,-0.146578,-1.386521,-0.369490,-0.143686,-0.983869,-0.057370,-0.311390,1.767312,-0.359733,-1.210991
1539,0.346078,1.307469,-0.026630,-0.344484,1.820002,-0.807671,0.044307,0.792166,1.484628,-0.311390,-1.424231,0.843412,-1.210991


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [11]:
# Define the objective function for Random Forest
def objective(trial):
    # Define hyperparameters for Random Forest
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 12, log=True),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
        "random_state": 42  # Set a fixed random seed for reproducibility
    }

    # Train the Random Forest model with the suggested hyperparameters
    model = RandomForestClassifier(**param)
    model.fit(x_train, y_train)

    # Predict on the validation set
    preds = model.predict(x_test)

    # Calculate accuracy
    acc = accuracy_score(y_test, preds)

    # Return the negative accuracy because Optuna minimizes the objective
    return 1.0 - acc  # Minimize the error (lower is better)

In [12]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 21:07:16,174] A new study created in memory with name: no-name-0598243a-dfcc-41a2-ad89-da01014632af
[I 2025-05-31 21:07:16,480] Trial 0 finished with value: 0.07327586206896552 and parameters: {'n_estimators': 193, 'max_depth': 3, 'min_samples_split': 3, 'min_samples_leaf': 7, 'max_features': None}. Best is trial 0 with value: 0.07327586206896552.
[I 2025-05-31 21:07:16,858] Trial 1 finished with value: 0.025862068965517238 and parameters: {'n_estimators': 178, 'max_depth': 7, 'min_samples_split': 8, 'min_samples_leaf': 9, 'max_features': None}. Best is trial 1 with value: 0.025862068965517238.
[I 2025-05-31 21:07:16,957] Trial 2 finished with value: 0.15086206896551724 and parameters: {'n_estimators': 143, 'max_depth': 3, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 1 with value: 0.025862068965517238.
[I 2025-05-31 21:07:17,091] Trial 3 finished with value: 0.030172413793103425 and parameters: {'n_estimators': 138, 'max_depth': 10

Best hyperparameters:
{'n_estimators': 164, 'max_depth': 12, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': None}
Best accuracy:
0.978448275862069


In [13]:
# Get the data
best_model = RandomForestClassifier(**study.best_params)
best_model.fit(x_train, y_train)

# Predict
preds = best_model.predict(x_test)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")



Final evaluation on test set:
Accuracy: 96.98
Precision: 96.30
Recall: 91.23
F1 Score: 93.69
